# Лабораторная работа 1 (KNN)

## Пояснения к выбору начальных условий

### Регрессия

#### Датасет:
Для решения задачи регрессии был выбран датасет "Laptop Prices" ([link](https://www.kaggle.com/datasets/owm4096/laptop-prices)). 

#### Метрики:

Для оценки модели были выбраны следующие метрики:

1. Mean Absolute Error (MAE) - Среднее абсолютное отклонение
2. Mean Squared Error (MSE) - Среднеквадратичное отклонение
3. R-squared ($R^2$) – Коэффициент детерминации

### Классификация

#### Датасет:

Для решения задачи классификации был выбран датасет "AIDS Virus Infection Prediction" ([link](https://www.kaggle.com/datasets/aadarshvelu/aids-virus-infection-prediction)). 

#### Метрики:

Для оценки модели были выбраны следующие метрики:

1. Accuracy – Точность
2. Precision – Точность предсказания для одного класса
3. Recall – Полнота
4. F1-score – Среднее гармоническое между Precision и Recall

#### Импортируем библиотеки

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from annoy import AnnoyIndex
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error,accuracy_score, precision_score, recall_score, f1_score
from collections import Counter
from utils import regression_cross_validate, display_metrics_table, classification_cross_validate, display_metrics_classification_table

In [2]:
import warnings
warnings.filterwarnings("ignore")

### Regression

#### 1. Обработка данных

In [3]:
df = pd.read_csv('data/laptop_prices.csv')
df.head()

,Company,Product,TypeName,Inches,Ram,OS,Weight,Price_euros,Screen,ScreenW,...,RetinaDisplay,CPU_company,CPU_freq,CPU_model,PrimaryStorage,SecondaryStorage,PrimaryStorageType,SecondaryStorageType,GPU_company,GPU_model
0,Apple,MacBook Pro,Ultrabook,13.3,8,macOS,1.37,1339.69,Standard,2560,...,Yes,Intel,2.3,Core i5,128,0,SSD,No,Intel,Iris Plus Graphics 640
1,Apple,Macbook Air,Ultrabook,13.3,8,macOS,1.34,898.94,Standard,1440,...,No,Intel,1.8,Core i5,128,0,Flash Storage,No,Intel,HD Graphics 6000
2,HP,250 G6,Notebook,15.6,8,No OS,1.86,575.00,Full HD,1920,...,No,Intel,2.5,Core i5 7200U,256,0,SSD,No,Intel,HD Graphics 620
3,Apple,MacBook Pro,Ultrabook,15.4,16,macOS,1.83,2537.45,Standard,2880,...,Yes,Intel,2.7,Core i7,512,0,SSD,No,AMD,Radeon Pro 455
4,Apple,MacBook Pro,Ultrabook,13.3,8,macOS,1.37,1803.60,Standard,2560,...,Yes,Intel,3.1,Core i5,256,0,SSD,No,Intel,Iris Plus Graphics 650


In [4]:
features, target = df.drop(columns=['Price_euros']), df['Price_euros']

num_cols= features.select_dtypes(include=[np.number]).columns  # Численные признаки
cat_cols = features.select_dtypes(exclude=[np.number]).columns  # Категориальные признаки

train_features, test_features, train_target, test_target = train_test_split(features, target, random_state=42)

ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value',
                    unknown_value=-1)

train_features[cat_cols] = ordinal_encoder.fit_transform(train_features[cat_cols])
test_features[cat_cols] = ordinal_encoder.transform(test_features[cat_cols])

most_frequent_imputer = SimpleImputer(strategy='most_frequent') 
train_features= pd.DataFrame(most_frequent_imputer.fit_transform(train_features), columns=train_features.columns)
test_features = pd.DataFrame(most_frequent_imputer.transform(test_features), columns=test_features.columns)

#### 2. Построение бейзлайна 

Для оценки модели будем использовать метод кросс валидации, который позволяет более качественно оценить полученные метрики.

Построим модель используя реализацию из sklearn

In [5]:
regression_metrics = regression_cross_validate(KNeighborsRegressor, train_features.to_numpy(), train_target.to_numpy(), n_folds=5, n_neighbors=3)
display_metrics_table(*regression_metrics)

knn = KNeighborsRegressor(n_neighbors=3)
knn.fit(train_features, train_target)

predicted_target = knn.predict(test_features)

# Метрики
mse = mean_squared_error(test_target, predicted_target)
mae = mean_absolute_error(test_target, predicted_target)
r2 = r2_score(test_target, predicted_target)

# Вывод метрик
print("\n=== Результаты на Тесте ===")
print(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")
print(f"Средняя абсолютная ошибка (MAE): {mae:.2f}")
print(f"Коэффициент детерминации (R^2): {r2:.2f}")

| Metric   |          Mean |     Std Dev |
|:---------|--------------:|------------:|
| MAE      |    285.831    |    19.1653  |
| MSE      | 217029        | 47245.6     |
| R2       |      0.562415 |     0.05428 |

=== Результаты на Тесте ===
Среднеквадратичная ошибка (MSE): 301414.49
Средняя абсолютная ошибка (MAE): 349.37
Коэффициент детерминации (R^2): 0.38


Значение метрики $R^2$ составляет около 0.38, что указывает на то, что модель объясняет примерно 38% вариации данных. Кроме того, среднее абсолютное значение ошибки (MAE) равно 349.37, что характеризует точность предсказаний модели в абсолютных единицах (в среднем предсказания ошибаются на 349 евро).

#### 3. Формулировка гипотез

Сформулируем несколько гипотез, которые могут помочь улучшить качество модели

1) Поменять Encoder категориальных признаков с `LabelEncoder` на `OneHotEncoder`
2) Отмасштабировать численные признаки
3) Увеличить число соседей

In [6]:
onehot = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')

encoded_train_data = onehot.fit_transform(train_features[cat_cols])
encoded_test_data = onehot.transform(test_features[cat_cols])

encoded_df = pd.DataFrame(encoded_train_data, columns=onehot.get_feature_names_out(cat_cols))
updated_train_features = train_features.drop(columns=cat_cols).reset_index(drop=True)
updated_train_features = pd.concat([updated_train_features, encoded_df], axis=1)

encoded_df = pd.DataFrame(encoded_test_data, columns=onehot.get_feature_names_out(cat_cols))
updated_test_features = test_features.drop(columns=cat_cols).reset_index(drop=True)
updated_test_features = pd.concat([updated_test_features, encoded_df], axis=1)


In [7]:
scaler = StandardScaler()
updated_train_features[num_cols] = scaler.fit_transform(train_features[num_cols])
updated_test_features[num_cols] = scaler.transform(test_features[num_cols])

In [8]:
regression_metrics = regression_cross_validate(KNeighborsRegressor, updated_train_features.to_numpy(), train_target.to_numpy(), n_folds=5, n_neighbors=5)
display_metrics_table(*regression_metrics)

knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(updated_train_features, train_target)

predicted_target = knn.predict(updated_test_features)

# Метрики
mse = mean_squared_error(test_target, predicted_target)
mae = mean_absolute_error(test_target, predicted_target)
r2 = r2_score(test_target, predicted_target)

# Вывод метрик
print("\n=== Результаты на Тесте ===")
print(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")
print(f"Средняя абсолютная ошибка (MAE): {mae:.2f}")
print(f"Коэффициент детерминации (R^2): {r2:.2f}")

| Metric   |          Mean |       Std Dev |
|:---------|--------------:|--------------:|
| MAE      |    203.488    |    11.6162    |
| MSE      | 102121        | 10468.3       |
| R2       |      0.791057 |     0.0227702 |

=== Результаты на Тесте ===
Среднеквадратичная ошибка (MSE): 101359.24
Средняя абсолютная ошибка (MAE): 207.16
Коэффициент детерминации (R^2): 0.79


Можно заметить, что в среднем показатели метрик значительно улучшились по сравнению с предыдущими результатами. Анализ результатов на тестовой выборке также демонстрирует существенные улучшения, что подтверждает эффективность проведённых модификаций.

#### 4. Реализация своего класса

In [9]:
class CustomKNNRegressor:
    def __init__(self, n_neighbors=5):
        self.n_neighbors = n_neighbors
    
    def fit(self, features, target):
        self.train_features= np.array(features)
        self.train_target = np.array(target)
    
    def _calculate_distance(self, x1, x2):
        return np.sqrt(np.sum((x1 - x2) ** 2))
    
    def predict(self, features):
        features = np.array(features)
        predictions = []
        
        for x in features:
            distances = np.array([self._calculate_distance(x, x_train) for x_train in self.train_features])
            nearest_indices = distances.argsort()[:self.n_neighbors]
            nearest_neighbors_values = self.train_target[nearest_indices]
            predictions.append(nearest_neighbors_values.mean())
        
        return np.array(predictions)

In [10]:
regression_metrics = regression_cross_validate(CustomKNNRegressor, train_features.to_numpy(), train_target.to_numpy(), n_folds=5, n_neighbors=3)
display_metrics_table(*regression_metrics)

knn = CustomKNNRegressor(n_neighbors=3)
knn.fit(train_features, train_target)

predicted_target = knn.predict(test_features)

# Метрики
mse = mean_squared_error(test_target, predicted_target)
mae = mean_absolute_error(test_target, predicted_target)
r2 = r2_score(test_target, predicted_target)

# Вывод метрик
print("\n=== Результаты на Тесте ===")
print(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")
print(f"Средняя абсолютная ошибка (MAE): {mae:.2f}")
print(f"Коэффициент детерминации (R^2): {r2:.2f}")

| Metric   |          Mean |       Std Dev |
|:---------|--------------:|--------------:|
| MAE      |    285.928    |    19.2828    |
| MSE      | 217053        | 47253.6       |
| R2       |      0.562367 |     0.0543076 |

=== Результаты на Тесте ===
Среднеквадратичная ошибка (MSE): 301308.79
Средняя абсолютная ошибка (MAE): 349.13
Коэффициент детерминации (R^2): 0.38


In [11]:
regression_metrics = regression_cross_validate(CustomKNNRegressor, updated_train_features.to_numpy(), train_target.to_numpy(), n_folds=5, n_neighbors=5)
display_metrics_table(*regression_metrics)

knn = CustomKNNRegressor(n_neighbors=5)
knn.fit(updated_train_features, train_target)

predicted_target = knn.predict(updated_test_features)

# Метрики
mse = mean_squared_error(test_target, predicted_target)
mae = mean_absolute_error(test_target, predicted_target)
r2 = r2_score(test_target, predicted_target)

# Вывод метрик
print("\n=== Результаты на Тесте ===")
print(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")
print(f"Средняя абсолютная ошибка (MAE): {mae:.2f}")
print(f"Коэффициент детерминации (R^2): {r2:.2f}")

| Metric   |          Mean |       Std Dev |
|:---------|--------------:|--------------:|
| MAE      |    203.892    |    11.6598    |
| MSE      | 102460        | 10417.9       |
| R2       |      0.790343 |     0.0228555 |

=== Результаты на Тесте ===
Среднеквадратичная ошибка (MSE): 101052.34
Средняя абсолютная ошибка (MAE): 206.55
Коэффициент детерминации (R^2): 0.79


Результаты демонстрируют, что разработанная собственная реализация модели в целом обеспечивает уровень качества, сопоставимый с результатами, достигаемыми стандартной моделью из библиотеки `sklearn`.

### Classification

#### 1. Обработка данных

In [12]:
df = pd.read_csv('data/AIDS_Classification.csv')
df.head()

,time,trt,age,wtkg,hemo,homo,drugs,karnof,oprior,z30,...,str2,strat,symptom,treat,offtrt,cd40,cd420,cd80,cd820,infected
0,948,2,48,89.8128,0,0,0,100,0,0,...,0,1,0,1,0,422,477,566,324,0
1,1002,3,61,49.4424,0,0,0,90,0,1,...,1,3,0,1,0,162,218,392,564,1
2,961,3,45,88.4520,0,1,1,90,0,1,...,1,3,0,1,1,326,274,2063,1893,0
3,1166,3,47,85.2768,0,1,0,100,0,1,...,1,3,0,1,0,287,394,1590,966,0
4,1090,0,43,66.6792,0,1,0,100,0,1,...,1,3,0,0,0,504,353,870,782,0


In [13]:
features, target = df.drop(columns=['infected']), df['infected']

train_features, test_features, train_target, test_target = train_test_split(features, target, random_state=42)

numeric_cols = features.select_dtypes(include=[np.number]).columns  # Численные признаки
cat_cols = features.select_dtypes(exclude=[np.number]).columns  # Категориальные признаки

ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value',
                    unknown_value=99)

train_features[cat_cols] = ordinal_encoder.fit_transform(train_features[cat_cols])
test_features[cat_cols] = ordinal_encoder.transform(test_features[cat_cols])

most_frequent_imputer = SimpleImputer(strategy='most_frequent') 
train_features= pd.DataFrame(most_frequent_imputer.fit_transform(train_features), columns=train_features.columns)
test_features = pd.DataFrame(most_frequent_imputer.transform(test_features), columns=test_features.columns)

#### 2. Построение бейзлайна 

In [14]:
classification_metrics = classification_cross_validate(KNeighborsClassifier, train_features.to_numpy(), train_target.to_numpy(), n_folds=5, n_neighbors=2)
display_metrics_classification_table(*classification_metrics)

knn = KNeighborsClassifier(n_neighbors=2)  # Выбираем количество соседей
knn.fit(train_features, train_target)

predicted_target = knn.predict(test_features)

accuracy_metric = accuracy_score(test_target, predicted_target)
precision_metric = precision_score(test_target, predicted_target, average='weighted')
recall_metric = recall_score(test_target, predicted_target, average='weighted')
f1_score_metric = f1_score(test_target, predicted_target, average='weighted')

# Выводим результаты
print("\n=== Результаты на Тесте ===")
print(f"1. Accuracy: {accuracy_metric:.2%}")
print(f"2. Precision: {precision_metric:.2%}")
print(f"3. Recall: {recall_metric:.2%}")
print(f"4. F1-score: {f1_score_metric:.2%}")

| Metric    |     Mean |   Std Dev |
|:----------|---------:|----------:|
| Accuracy  | 0.797385 | 0.0308157 |
| Precision | 0.778979 | 0.0370002 |
| Recall    | 0.797385 | 0.0308157 |
| F1-score  | 0.77263  | 0.0352112 |

=== Результаты на Тесте ===
1. Accuracy: 80.56%
2. Precision: 78.99%
3. Recall: 80.56%
4. F1-score: 78.42%


#### 3. Формулировка гипотез

Сформулируем несколько гипотез, которые могут помочь улучшить качество модели

1) Поменять Encoder категориальных признаков с `LabelEncoder` на `OneHotEncoder`
2) Увеличить число соседей

In [15]:
onehot = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')

encoded_train_data = onehot.fit_transform(train_features[cat_cols])
encoded_test_data = onehot.transform(test_features[cat_cols])

encoded_df = pd.DataFrame(encoded_train_data, columns=onehot.get_feature_names_out(cat_cols))
updated_train_features = train_features.drop(columns=cat_cols).reset_index(drop=True)
updated_train_features = pd.concat([updated_train_features, encoded_df], axis=1)

encoded_df = pd.DataFrame(encoded_test_data, columns=onehot.get_feature_names_out(cat_cols))
updated_test_features = test_features.drop(columns=cat_cols).reset_index(drop=True)
updated_test_features = pd.concat([updated_test_features, encoded_df], axis=1)

In [16]:
classification_metrics = classification_cross_validate(KNeighborsClassifier, updated_train_features.to_numpy(), train_target.to_numpy(), n_folds=5, n_neighbors=5)
display_metrics_classification_table(*classification_metrics)

knn = KNeighborsClassifier(n_neighbors=5)  # Выбираем количество соседей
knn.fit(updated_train_features, train_target)

predicted_target = knn.predict(updated_test_features)

accuracy_metric = accuracy_score(test_target, predicted_target)
precision_metric = precision_score(test_target, predicted_target, average='weighted')
recall_metric = recall_score(test_target, predicted_target, average='weighted')
f1_score_metric = f1_score(test_target, predicted_target, average='weighted')

# Выводим результаты
print("\n=== Результаты на Тесте ===")
print(f"1. Accuracy: {accuracy_metric:.2%}")
print(f"2. Precision: {precision_metric:.2%}")
print(f"3. Recall: {recall_metric:.2%}")
print(f"4. F1-score: {f1_score_metric:.2%}")

| Metric    |     Mean |    Std Dev |
|:----------|---------:|-----------:|
| Accuracy  | 0.832921 | 0.00569355 |
| Precision | 0.825686 | 0.00687814 |
| Recall    | 0.832921 | 0.00569355 |
| F1-score  | 0.826611 | 0.00653782 |

=== Результаты на Тесте ===
1. Accuracy: 83.36%
2. Precision: 82.46%
3. Recall: 83.36%
4. F1-score: 82.55%


Видно, что в среднем показатели метрик заметно улучшились. Результаты, полученные на тестовой выборке, также свидетельствуют о значительных приростах.

#### 4. Реализация своего класса

In [17]:
class CustomKNNClassifier:
    def __init__(self, n_neighbors=5):
        self.n_neighbors = n_neighbors
        self.n_trees = 10
    
    def fit(self, features, target):
        self.train_features= np.array(features)
        self.train_target = np.array(target)
        self.n_features = self.train_features.shape[1]

        # Создаём индекс Annoy
        self.index = AnnoyIndex(self.n_features)
        for i, x in enumerate(self.train_features):
            self.index.add_item(i, x)
        
        # Строим деревья
        self.index.build(self.n_trees)
    
    
    def predict(self, features):
        features = np.array(features)
        predictions = []
        for x in features:
            nearest_indices = self.index.get_nns_by_vector(x, self.n_neighbors)
            
            nearest_labels = self.train_target[nearest_indices]
            
            most_common_class = Counter(nearest_labels).most_common(1)[0][0]
            predictions.append(most_common_class)
        
        return np.array(predictions)

In [18]:
classification_metrics = classification_cross_validate(CustomKNNClassifier, train_features.to_numpy(), train_target.to_numpy(), n_folds=5, n_neighbors=3)
display_metrics_classification_table(*classification_metrics)

knn = CustomKNNClassifier(n_neighbors=3)
knn.fit(train_features, train_target)

predicted_target = knn.predict(test_features)

accuracy_metric = accuracy_score(test_target, predicted_target)
precision_metric = precision_score(test_target, predicted_target, average='weighted')
recall_metric = recall_score(test_target, predicted_target, average='weighted')
f1_score_metric = f1_score(test_target, predicted_target, average='weighted')

# Выводим результаты
print("\n=== Результаты на Тесте ===")
print(f"1. Accuracy: {accuracy_metric:.2%}")
print(f"2. Precision: {precision_metric:.2%}")
print(f"3. Recall: {recall_metric:.2%}")
print(f"4. F1-score: {f1_score_metric:.2%}")

| Metric    |     Mean |   Std Dev |
|:----------|---------:|----------:|
| Accuracy  | 0.786791 | 0.0231232 |
| Precision | 0.772564 | 0.0272339 |
| Recall    | 0.786791 | 0.0231232 |
| F1-score  | 0.77609  | 0.0272324 |

=== Результаты на Тесте ===
1. Accuracy: 77.20%
2. Precision: 75.07%
3. Recall: 77.20%
4. F1-score: 75.56%


In [19]:
classification_metrics = classification_cross_validate(CustomKNNClassifier, updated_train_features.to_numpy(), train_target.to_numpy(), n_folds=5, n_neighbors=5)
display_metrics_classification_table(*classification_metrics)

knn = CustomKNNClassifier(n_neighbors=5)
knn.fit(updated_train_features, train_target)

predicted_target = knn.predict(updated_test_features)

accuracy_metric = accuracy_score(test_target, predicted_target)
precision_metric = precision_score(test_target, predicted_target, average='weighted')
recall_metric = recall_score(test_target, predicted_target, average='weighted')
f1_score_metric = f1_score(test_target, predicted_target, average='weighted')

# Выводим результаты
print("\n=== Результаты на Тесте ===")
print(f"1. Accuracy: {accuracy_metric:.2%}")
print(f"2. Precision: {precision_metric:.2%}")
print(f"3. Recall: {recall_metric:.2%}")
print(f"4. F1-score: {f1_score_metric:.2%}")

| Metric    |     Mean |   Std Dev |
|:----------|---------:|----------:|
| Accuracy  | 0.799877 | 0.0209608 |
| Precision | 0.785339 | 0.0227115 |
| Recall    | 0.799877 | 0.0209608 |
| F1-score  | 0.786846 | 0.0238761 |

=== Результаты на Тесте ===
1. Accuracy: 78.32%
2. Precision: 75.87%
3. Recall: 78.32%
4. F1-score: 75.32%


Результаты показывают, что собственная реализация модели в среднем уступает по качеству модели из `sklearn`, однако демонстрирует приросты при использовании улучшенного бейзлайна.

### Заключение

Изменения, такие как OneHotEncoding и увеличение количества соседей в алгоритме поиска ближайших соседей, способствуют повышению точности модели. Эксперименты показывают, что собственная реализация и модели из библиотеки sklearn обеспечивают сопоставимые результаты.

| Модель                    |      MSE  |        MAE |      $R^2$ |
|:--------------------------|----------:|-----------:|-----------:|
| Sklearn (до улучшения)    | 301414.49 | 349.37     |  0.38      |
| Sklearn (после улучшения) | 101359.24 | 207.16     |  0.79      |
| Собственная имплементация (до улучшения)   | 301308.79 |  349.13   |  0.38     |
| Собственная имплементация (после улучшения)| 101052.34  | 206.55    |  0.79      |

| Модель                    |  Accuracy |  Precision |     Recall |    F1-scor |
|:--------------------------|----------:|-----------:|-----------:|-----------:|
| Sklearn (до улучшения)    |   80.56%  |  78.99%   |  80.56%   |  78.42%    |
| Sklearn (после улучшения) |   83.36%  |   82.46%   |  83.36%   |  82.55%    |
| Собственная имплементация (до улучшения)   |   77.20%  |   75.07%   |  77.20%    |  75.56%    |
| Собственная имплементация (после улучшения)|   78.32%  |   75.87%   |  78.32%    |  75.32%    |